In [1]:
!nvcc --version 

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:18:20_PST_2022
Cuda compilation tools, release 11.6, V11.6.124
Build cuda_11.6.r11.6/compiler.31057947_0


In [ ]:
!pip install --upgrade pip
# Installs the wheel compatible with CUDA 11 and cuDNN 8.6 or newer.
# Note: wheels only available on linux.
!pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

In [ ]:
!pip install flax optax tensorflow 

In [4]:
!pip install --upgrade pip

# Installs the wheel compatible with Cuda >= 11.4 and cudnn >= 8.2
!pip install "jax[cuda11_cudnn82]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
  Using cached https://storage.googleapis.com/jax-releases/cuda11/jaxlib-0.4.4%2Bcuda11.cudnn82-cp38-cp38-manylinux2014_x86_64.whl (154.5 MB)
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.4+cuda11.cudnn86
    Uninstalling jaxlib-0.4.4+cuda11.cudnn86:
      Successfully uninstalled jaxlib-0.4.4+cuda11.cudnn86


In [5]:
!rm -rf datasets
!rm -rf unet
!git clone https://github.com/r21gh/unet.git
!mv unet/datasets .
!rm -rf unet
!rm -rf checkpoints


Cloning into 'unet'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 126 (delta 39), reused 117 (delta 34), pack-reused 0
Receiving objects: 100% (126/126), 5.49 MiB | 19.60 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [6]:
# data_loader.py
import os, glob
import tensorflow as tf
from jax.lib import xla_bridge

PATH = 'datasets/'
BATCH_SIZE = 12
IMAGE_SIZE = 512

def read_train_data():
    x_files = tf.data.Dataset.list_files(PATH + "train/*.png")
    y_files = tf.data.Dataset.list_files(PATH + "mask/*.png")

    def read_image(x_filename, y_filename):
        x_image_string = tf.io.read_file(x_filename)
        y_image_string = tf.io.read_file(y_filename)

        x_image_decoded = tf.image.decode_png(x_image_string, channels=3)
        y_image_decoded = tf.image.decode_png(y_image_string, channels=1)

        x_image_resized = tf.image.resize(x_image_decoded, [IMAGE_SIZE, IMAGE_SIZE])
        y_image_resized = tf.image.resize(y_image_decoded, [IMAGE_SIZE, IMAGE_SIZE])

        x_image_norm = x_image_resized / 255
        y_image_norm = y_image_resized / 255

        return x_image_norm, y_image_norm

    dataset = tf.data.Dataset.zip((x_files, y_files))

    dataset = dataset.map(read_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(1000).batch(BATCH_SIZE)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset


def read_predict_data():
    def image_generator():
        for filename in glob.iglob(PATH + "test/*.png"):
            with open(filename, "rb") as f:
                image = tf.image.decode_png(f.read(), channels=3)
                image_resized = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
                image_norm = image_resized / 255.0
                yield image_norm

    images_dataset = tf.data.Dataset.from_generator(
        image_generator,
        output_types=tf.float32,
        output_shapes=(IMAGE_SIZE, IMAGE_SIZE, 3),
    )
    images_dataset = images_dataset.batch(BATCH_SIZE)
    images_dataset = images_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return images_dataset


def save_image(image):
    file_path = 'test/result.png'
    image = image * 255

    encode_image = tf.image.encode_png(image, format='rgb', quality=100)

    with open(file_path, 'wb') as fd:
        fd.write(encode_image)

In [7]:
# model.py
import jax
from jax import numpy as jnp
from flax import linen as fnn


class Encoder(fnn.Module):
    """
    This code defines a class called Encoder, 
    which is a component of a UNet architecture
    commonly used in image segmentation tasks. 
    The Encoder class takes as input an image 
    tensor and produces a set of skip connections, 
    which are used by the Decoder network to 
    generate a segmentation mask.

    The Encoder network consists of a series of 
    convolutional layers with increasing numbers 
    of channels and decreasing spatial dimensions.
    Each set of features is passed through batch 
    normalization and activation functions. The 
    network also includes max-pooling and dropout 
    layers in the higher-resolution feature maps 
    to reduce overfitting and improve generalization.

    The Encoder class has two class variables: 
    features and training. features is an integer 
    that specifies the number of channels in the 
    convolutional layers of the network. training 
    is a boolean that specifies whether the network
    is in training mode or not.

    The __call__ method of the Encoder class is defined
    using the @fnn.compact decorator, which is used in
    the JAX library to define neural networks in a 
    concise manner. The __call__ method takes the input
    image tensor x as input and returns a list of skip 
    connections as output.

    The Encoder network applies a series of convolutional 
    layers to the input image, producing a set of feature 
    maps with different spatial dimensions and channel sizes. 
    The network then applies batch normalization and ReLU 
    activation functions to each set of features. Max-pooling
    is used in the lower-resolution feature maps to reduce 
    the spatial dimensions and provide translation invariance.
    Dropout is used in the higher-resolution feature maps to 
    reduce overfitting and improve generalization.

    The output of the Encoder network is a list of feature maps 
    at multiple scales. The feature maps in the list are in 
    decreasing order of spatial dimensions and increasing order 
    of channel sizes. The list of feature maps is used as input 
    to the Decoder network, which upsamples and combines the 
    features to produce a segmentation mask.

    Overall, the Encoder class defines the feature extraction 
    and downsampling operations of the UNet architecture, 
    which are crucial for producing accurate segmentation 
    masks in image segmentation tasks.

    """
    features: int = 64
    training: bool = True

    @fnn.compact
    def __call__(self, x):
        skips = []
        for i in range(5):
            z = fnn.Conv(self.features * 2 ** i, kernel_size=(3, 3))(x)
            z = fnn.relu(z)
            z = fnn.Conv(self.features * 2 ** i, kernel_size=(3, 3))(z)
            z = fnn.BatchNorm(use_running_average=not self.training)(z)
            z = fnn.relu(z)
            if i < 4:
                x = fnn.max_pool(z, window_shape=(2, 2), strides=(2, 2))
            if i == 3 or i == 4:
                z = fnn.Dropout(0.5, deterministic=False)(z)
            skips.append(z)
            
        return skips


class Decoder(fnn.Module):
    """ This code defines a class called Decoder, 
        which is a component of a UNet architecture 
        commonly used in image segmentation tasks. 
        The Decoder class takes as input a set of 
        skip connections z_skips, which are produced 
        by the Encoder network and contain feature 
        maps at multiple scales.
        
        The Decoder class has two class variables: 
        features and training. features is an integer 
        that specifies the number of channels in the 
        convolutional layers of the network. 
        training is a boolean that specifies whether 
        the network is in training mode or not.
        
        The __call__ method of the Decoder class is
        defined using the @fnn.compact decorator, which 
        is used in the JAX library to define neural networks
        in a concise manner. The __call__ method takes 
        the set of skip connections z_skips as input and 
        returns a segmentation mask as output.

        The Decoder network first selects the last set of
        features from z_skips, and then iteratively upsamples
        and concatenates the feature maps from the other sets 
        of skip connections with the current set of features. 
        The concatenated features are then passed through a 
        series of convolutional layers and activation functions 
        to produce the final segmentation mask.

        The output of the Decoder network is a single-channel 
        tensor with the same spatial dimensions as the input 
        images. Each pixel in the output tensor represents the
        probability that the corresponding pixel in the input 
        image belongs to the object or class being segmented. 
        The sigmoid activation function is used to ensure that 
        the output values are in the range [0, 1].

        Overall, the Decoder class defines the upsampling and 
        feature combination operations of the UNet architecture, 
        which are crucial for producing accurate segmentation masks
        in image segmentation tasks.

    """
    features: int = 64
    training: bool = True

    @fnn.compact
    def __call__(self, z_skips):
        z = z_skips[-1]
        for i in range(4):
            z = jax.image.resize(z, 
                                 shape=(z.shape[0], z.shape[1] * 2, z.shape[2] * 2, z.shape[3]), 
                                 method='nearest')
            z = fnn.Conv(self.features * 8, kernel_size=(2, 2))(z)
            z = fnn.relu(z)
            z = jnp.concatenate([z_skips[-i-2], z], axis=3)
            z = fnn.Conv(self.features * 8, kernel_size=(3, 3))(z)
            z = fnn.relu(z)
            z = fnn.Conv(self.features * 8, kernel_size=(3, 3))(z)
            z = fnn.BatchNorm(use_running_average=not self.training)(z)
            z = fnn.relu(z)

        z = fnn.Conv(1, kernel_size=(1, 1))(z)
        z = fnn.sigmoid(z)

        return z


class UNet(fnn.Module):
    """This code defines a class called UNet, 
       which is a type of convolutional neural 
       network commonly used for image segmentation 
       tasks. The UNet architecture consists of 
       an encoder and a decoder network, which are 
       connected through skip connections.
       The class has two class variables: features 
       and training. features is an integer that 
       specifies the number of channels in the 
       convolutional layers of the network. training 
       is a boolean that specifies whether the 
       network is in training mode or not.

    Args:
        fnn (__call__): 
        The __call__ method of the UNet class is 
        defined using the @fnn.compact decorator, 
        which is used in the JAX library to define 
        neural networks in a concise manner.
        The __call__ method takes an input tensor x, 
        which is passed through the Encoder network 
        to produce a set of skip connections. 
        These skip connections are then passed through 
        the Decoder network to produce the final
        output tensor y.
        The Encoder network is responsible for 
        downsampling the input tensor and extracting 
        high-level features, while the Decoder network 
        is responsible for upsampling the features 
        and producing the final segmentation mask.

    Returns:
        _type_: 
        The return value of the __call__ method in 
        this code is a tensor y, which represents 
        the output of the UNet architecture after 
        processing the input tensor x. The exact
        shape and data type of y will depend on the 
        specific implementation of the Decoder 
        network and the nature of the image segmentation
        task that the UNet is being used for.
        In general, the output tensor y will likely 
        be a multi-channel tensor with the same spatial 
        dimensions as the input tensor x. Each channel 
        of y may represent a different segmentation class 
        or feature of the input image, depending on the 
        task at hand. The data type of y will likely 
        be a floating-point type, such as float32, 
        to allow for the computation of gradients 
        during backpropagation.
    
    Overall, this code defines a UNet architecture 
    that can be used for image segmentation tasks. 
    By changing the value of the features and 
    training variables, the architecture can be 
    customized to fit different input data and training regimes.
    """
    features: int = 64
    training: bool = True

    @fnn.compact
    def __call__(self, x):
        z_skips = Encoder(self.training)(x)
        y = Decoder(self.training)(z_skips)

        return y

In [8]:
# prediction.py
import time
from functools import partial
import jax
import optax

from jax import numpy as jnp
from flax.training.train_state import TrainState
from flax.training import checkpoints


CKPT_DIR = 'checkpoints'
IMAGE_SIZE = 512

def predict():
    data = read_predict_data()
    unet = UNet(training=False)

    init_rngs = {'params': jax.random.PRNGKey(0), 'dropout': jax.random.PRNGKey(1)}

    unet_variables = unet.init(init_rngs, jnp.ones([1, IMAGE_SIZE, IMAGE_SIZE, 3]))

    optimizer = optax.adam(learning_rate=0.001)

    train_state = CustomTrainState.create(apply_fn=unet.apply, params=unet_variables["params"], tx=optimizer,
                                          batch_stats=unet_variables["batch_stats"])

    checkpoints.restore_checkpoint(ckpt_dir=CKPT_DIR, target=train_state)

    pred, _ = train_state.apply_fn_with_bn({"params": train_state.params, "batch_stats": train_state.batch_stats}, data)

    save_image(pred)

In [9]:
# training.py
from functools import partial
import jax

from jax import numpy as jnp
from flax.training.train_state import TrainState
from flax.training import checkpoints

CKPT_DIR = 'checkpoints'
IMAGE_SIZE = 512


class CustomTrainState(TrainState):
    batch_stats: dict

    def apply_fn_with_bn(self, *args, **kwargs):
        output, mutated_vars = self.apply_fn(*args, **kwargs,
                                             mutable=["batch_stats"], rngs={'dropout': jax.random.PRNGKey(2)})
        new_batch_stats = mutated_vars["batch_stats"]
        return output, new_batch_stats

    def update_batch_stats(self, new_batch_stats):
        return self.replace(batch_stats=new_batch_stats)
    
@jax.jit
def dice_coef(y_true, y_pred):
    y_true = jnp.ravel(y_true)
    y_pred = jnp.ravel(y_pred)
    intersection = jnp.sum(y_true * y_pred)
    return 2.0 * intersection / (jnp.sum(y_true) + jnp.sum(y_pred) + 1)


@jax.jit
def dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

@partial(jax.jit, static_argnums=(3,))
def train_step(x, y, train_state, is_training=True):
    def loss_fn(params, batch_stats, is_training):
        y_pred, batch_stats = train_state.apply_fn_with_bn({"params": params, "batch_stats": batch_stats}, x)
        loss = dice_coef_loss(y, y_pred)

        return loss, batch_stats

    if is_training:
        grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
        (loss, batch_stats), grads = grad_fn(train_state.params, train_state.batch_stats, True)

        train_state = train_state.apply_gradients(grads=grads)
        train_state = train_state.update_batch_stats(batch_stats)
    else:
        loss, batch_stats = loss_fn(train_state.params, train_state.batch_stats, False)

    return loss, train_state

In [ ]:
# main.py
import time
from functools import partial
import jax
import optax

from jax import numpy as jnp
from flax.training.train_state import TrainState
from flax.training import checkpoints

CKPT_DIR = 'checkpoints'
IMAGE_SIZE = 512
NUM_EPOCHS = 20

def run():
    train_set = read_train_data()
    unet = UNet()

    init_rngs = {'params': jax.random.PRNGKey(0), 'dropout': jax.random.PRNGKey(1)}

    unet_variables = unet.init(init_rngs, jnp.ones([1, IMAGE_SIZE, IMAGE_SIZE, 3]))

    optimizer = optax.adam(learning_rate=0.001)

    train_state = CustomTrainState.create(apply_fn=unet.apply, params=unet_variables["params"], tx=optimizer,
                                          batch_stats=unet_variables["batch_stats"])

    checkpoints.save_checkpoint(ckpt_dir=CKPT_DIR, target=train_state, step=0, overwrite=True)

    for e in range(NUM_EPOCHS):
        loss_avg = 0
        tic = time.time()
        for x, y in train_set.as_numpy_iterator():
            loss, train_state = train_step(x, y, train_state, True)
            loss_avg += loss

        loss_avg /= len(train_set)
        elapsed = time.time() - tic
        print(f"epoch: {e}, loss: {loss_avg:0.2f}, elapased: {elapsed:0.2f}")

    predict()

if __name__ == '__main__':
    run()